# Churn Modelling

In [ ]:
## Importing the Libraries
import pandas as pd
import numpy as np

## Let us load the dataset

In [ ]:
dataset = pd.read_csv('../input/churn-modelling/Churn_Modelling.csv')

## Let us have a look at the dataset

In [ ]:
dataset.head()

In [ ]:
dataset.info()

## Let us do some stats on the dataset

In [ ]:
dataset.describe()

## Looks like we don't have any missing values, however let us confirm that below

In [ ]:
dataset.isnull().sum()

## Time to split the data into set of Independent and Dependent variables

In [ ]:
X = dataset.iloc[:, 3:13]
y = dataset.iloc[:, 13]

## We can cleary see from the describe output that few columns don't make much interesting for our analysis
## We have discarded RowNumber, CustomerId and Surname while splitting the data
## Now, we will replace Geography and Gender into dummies

In [ ]:
geography = pd.get_dummies(X['Geography'], drop_first = True)
gender = pd.get_dummies(X['Gender'], drop_first = True)

## Let us concatenate the dataset of dummies with the original dataset

In [ ]:
X = pd.concat([X, geography, gender], axis = 1)

## Now we do not need the Geography and Gender raw data in our dataset, we will drop/delete them

In [ ]:
X = X.drop(['Geography', 'Gender'], axis = 1)

## Let us do training testing splits

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 101)

## We will do scaling of the features

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Let us do some Hyperparameter Optimization
## Importing necessary Libraries

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Embedding, Flatten, LeakyReLU, BatchNormalization, Dropout
from tensorflow.keras.activations import relu, sigmoid

## Function for creating the model that takes layers and activation functions as arguments

In [ ]:
def create_model(layers, activation):
    model = Sequential()
    for i, nodes in enumerate(layers):
        if i==0:
            model.add(Dense(nodes,input_dim=X_train.shape[1]))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
        else:
            model.add(Dense(nodes))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
            
    model.add(Dense(units = 1, kernel_initializer= 'glorot_uniform', activation = 'sigmoid')) 
    
    model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
    return model
model = KerasClassifier(build_fn=create_model, verbose=0)

In [ ]:
model =  KerasClassifier(build_fn = create_model, verbose = 0)

layers = [[20], [40, 20], [45, 30, 15]]
activations = ['sigmoid', 'relu']
param_grid = dict(layers=layers, activation = activations, batch_size = [128, 256], epochs = [30])
grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = 5)

In [ ]:
grid_result = grid.fit(X_train, y_train)

## There is some issue with wrappers class of keras
## We have run the same code in Google Colab and it worked
## So, we will just take the best parameters and then create our neural network
## The best parameters are : activation:relu, batch_size: 128, epochs = 30, layers: 40, 20

In [ ]:
ann_model = Sequential()
ann_model.add(Dense(units = 40, activation = 'relu'))
ann_model.add(Dense(units = 20, activation = 'relu'))


In [ ]:
ann_model.add(Dense(units = 1, activation = 'sigmoid'))

In [ ]:
ann_model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
ann_model.fit(X_train, y_train, epochs = 30, batch_size = 128)

In [ ]:
y_pred = ann_model.predict(X_test)
y_pred = y_pred > 0.5

## Let us see our confusion matrix and classification report

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
cm = confusion_matrix(y_pred, y_test)
accuracy = accuracy_score(y_pred, y_test)
print(classification_report(y_test, y_pred))

In [ ]:
print(cm)

In [ ]:
accuracy